In [2]:
import pandas as pd


In [3]:
df_groundtruth = pd.read_csv('../ClaimBuster_Datasets/datasets/groundtruth.csv')
df_all_sentences = pd.read_csv('../ClaimBuster_Datasets/datasets/all_sentences.csv')
df_crowdsourced = pd.read_csv('../ClaimBuster_Datasets/datasets/crowdsourced.csv')
df_crwd_grnd = pd.concat([df_groundtruth,df_crowdsourced],axis=0)

df_crwd_grnd.sort_values('Sentence_id').head()


,Sentence_id,Text,Speaker,Speaker_title,Speaker_party,File_id,Length,Line_number,Sentiment,Verdict
0,16,I think we've seen a deterioration of values.,George Bush,Vice President,REPUBLICAN,1988-09-25.txt,8,16,0.000000,-1
1,17,I think for a while as a nation we condoned th...,George Bush,Vice President,REPUBLICAN,1988-09-25.txt,16,17,-0.456018,-1
2,18,"For a while, as I recall, it even seems to me ...",George Bush,Vice President,REPUBLICAN,1988-09-25.txt,29,18,-0.805547,-1
3,19,"So we've seen a deterioration in values, and o...",George Bush,Vice President,REPUBLICAN,1988-09-25.txt,35,19,0.698942,-1
4,20,"We got away, we got into this feeling that val...",George Bush,Vice President,REPUBLICAN,1988-09-25.txt,15,20,0.000000,-1


In [4]:
sentence_id_with_v = df_crwd_grnd['Sentence_id'].values

In [5]:
df_without_label = df_all_sentences[~((df_all_sentences['Speaker'] == 'Information') | (df_all_sentences['Speaker_title'].isna()) | (df_all_sentences['Sentence_id'].isin(sentence_id_with_v))) ]
df_without_label['Verdict'] = 7
df_without_label.head()

C:\Users\mike1\AppData\Local\Temp\ipykernel_19180\631960669.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_without_label['Verdict'] = 7


,Sentence_id,Text,Speaker,Speaker_title,Speaker_party,Speaker_role,File_id,Length,Line_number,Sentiment,Verdict
27,28,And it's bad.,George Bush,Vice President,REPUBLICAN,Candidate,1988-09-25.txt,3,28,-0.736546,7
101,102,"Governor Dukakis, another troublesome issue fo...",John Mashek,"Correspondent, Atlanta Journal-Constitution",NaN,Questioner,1988-09-25.txt,14,102,-0.695606,7
102,103,"In a Dukakis administration, you say taxes wil...",John Mashek,"Correspondent, Atlanta Journal-Constitution",NaN,Questioner,1988-09-25.txt,15,103,-0.567921,7
103,104,Would you identify for us then please three sp...,John Mashek,"Correspondent, Atlanta Journal-Constitution",NaN,Questioner,1988-09-25.txt,21,104,-0.572394,7
121,122,(Laughter),Michael Dukakis,Governor,DEMOCRAT,Candidate,1988-09-25.txt,1,122,0.000000,7


In [17]:
#concat all datasets
df_full = pd.concat([df_crwd_grnd,df_without_label],axis=0)


df_full = df_full.sort_values('Sentence_id')
df_full.reset_index(drop=True,inplace=True)

In [20]:
def generate_word_dataset(df,presentence = 1,postsentence = 1):
    dataset = []
    label = []
    for index,row in df.iterrows():
        if index == 0 or index == len(df)-1:
            continue
        past = df.iloc[index-1]['Text']
        curr = row['Text']
        future = df.iloc[index+1]['Text']    
        if row['File_id'] != df.iloc[index-1]['File_id']:
            past = "EMPTY"
        if row['File_id'] != df.iloc[index+1]['File_id']:
            future = "EMPTY"
        dataset.append([past,curr,future])
        label.append(row['Verdict'])
    
    return dataset,label

data_text, labels = generate_word_dataset(df_full)

In [24]:
data_text[0],data_text[1] #can see each training sample has one past sentence curr sentence and one future sentence

(["I think we've seen a deterioration of values.",
  'I think for a while as a nation we condoned those things we should have condemned.',
  "For a while, as I recall, it even seems to me that there was talk of legalizing or decriminalizing marijuana and other drugs, and I think that's all wrong."],
 ['I think for a while as a nation we condoned those things we should have condemned.',
  "For a while, as I recall, it even seems to me that there was talk of legalizing or decriminalizing marijuana and other drugs, and I think that's all wrong.",
  "So we've seen a deterioration in values, and one of the things that I think we should do about it in terms of cause is to instill values into the young people in our schools."])